In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import websocket
import zmq
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
f = make_subplots(rows=3, cols=1, shared_xaxes=True, row_width=[1, 1, 3], subplot_titles=["Strategy return", "Position", "Drawdown"])
f.append_trace(go.Scatter(name='BTC-USDT return'), row=1, col=1)
f.append_trace(go.Scatter(name='Strategy return'), row=1, col=1)
f.append_trace(go.Scatter(name='Strategy Max'), row=1, col=1)
f.append_trace(go.Scatter(name='Position'), row=2, col=1)
f.append_trace(go.Scatter(name='Drawdown'), row=3, col=1)

f.update_layout(height=600, title="BTC-USDT Momentum strategy")
fig = go.FigureWidget(f)
fig

In [ ]:
def compute_statistics():
    df['return'] = np.log(df['price'] / df['price'].shift(1))
    df['balance_return'] = np.log(df['balance'] / df['balance'].shift(1))
    df['cum_returns'] = df['return'].cumsum().apply(np.exp)
    df['cum_strategy'] = df['balance_return'].cumsum().apply(np.exp)
    df['cum_max'] = df['cum_strategy'].cummax()
    df['drawdown'] = df['cum_max'] - df['cum_strategy']

In [ ]:
def update_chart():
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[3].x = df.index
    fig.data[4].x = df.index
    fig.data[0].y = df['cum_returns']
    fig.data[1].y = df['cum_strategy']
    fig.data[2].y = df['cum_max']
    fig.data[3].y = df['position']
    fig.data[4].y = df['drawdown']

In [ ]:
# Load historical ticks from the start of the trading session
headers = ["Date", "price", "position", "balance"]
file = './logs/trading-logs.csv'
df = pd.read_csv(file, parse_dates=True, index_col=0, header=None, names=headers)

compute_statistics()
update_chart()

In [ ]:
context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect('tcp://0.0.0.0:5555')
socket.setsockopt_string(zmq.SUBSCRIBE, "LOGS:")

while True:
    # receive and parse message to append it to a DataFrame
    msg = socket.recv_string()
    _, raw = msg.split("LOGS:")
    data = json.loads(raw)
    date = dt.datetime.fromtimestamp(int(data['timestamp']))
    new_df = pd.DataFrame(data['raw'], index=[date])
    df = pd.concat([df, new_df])

    compute_statistics()
    update_chart()